### User가 지정한 csv index 값을 받아, 해당 index의 데이터를 기반으로 최적화 수행

In [1]:
import surrogate_model
import argparse
import numpy as np
import pandas as pd
import os
import search_model
from src.datasets.data_loader import load_data
import src.dynamic_pipeline as dynamic_pipeline


c:\Users\seong\conda3\envs\hack\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def concrete():
    df = pd.read_csv('./data/concrete.csv')

    target = 'strength'
    user_request_target = 40.0

    control_name = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age']
    importance={'cement': 1,
                    'water': 2,
    }
    optimize={'cement': 'minimize',
                'water': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [3]:
def dvm():
    df = pd.read_csv('./data/dvm_merged_df.csv')

    target = 'Annual_revenue'
    user_request_target = 2485797880

    control_name = ['Entry_price', 'Fuel_type', 'Bodytype', 'Top_speed', 'Gearbox']
    # control_name = [col for col in df.columns if col != target]
    importance={'Entry_price': 1,
                    'Top_speed': 2,
    }
    optimize={'Entry_price': 'minimize',
                'Top_speed': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [4]:
def ecommerce():
    df = pd.read_csv('./data/ecommerce_sampled_10000.csv')

    target = 'revenue'
    user_request_target = 50000000

    control_name = ['discount_rate']
    # control_name = [col for col in df.columns if col != target]
    importance={'discount_rate': 1,
    }
    optimize={'discount_rate': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [5]:
def melb():
    df = pd.read_csv('./data/melb_data.csv')

    target = 'Price'
    user_request_target = 7000000

    control_name = ['Suburb', 'Rooms', 'Landsize']
    # control_name = [col for col in df.columns if col != target]
    importance={'Landsize': 1,
            'Rooms': 2,
    }
    optimize={'Landsize': 'minimize',
            'Rooms': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [6]:
def employee():
    df = pd.read_csv('./data/employee_attrition_dataset.csv')

    target = 'Attrition'
    user_request_target = 'No'

    control_name = ['Job_Role', 'Hourly_Rate', 'Average_Hours_Worked_Per_Week', 'Relationship_with_Manager', 'Job_Involvement', 'Number_of_Companies_Worked']
    # control_name = [col for col in df.columns if col != target]
    importance={'Hourly_Rate': 1,
                'Average_Hours_Worked_Per_Week': 2,
    }
    optimize={'Hourly_Rate': 'minimize',
            'Average_Hours_Worked_Per_Week': 'maximize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [7]:
df, target, user_request_target, control_name, importance, optimize = concrete() # concrete(), dvm(), ecommerce(), melb(), employee()
model = 'tabpfn' # 'catboost','tabpfn'

In [8]:
ret_dict = dynamic_pipeline.preprocess_dynamic(df.copy())
ret_dict['processed_df'].to_csv('./temp.csv', index=False)

제거된 컬럼이 없습니다.
cement: 비대칭 분포로 확인됨. IQR 방식 적용.
slag: 비대칭 분포로 확인됨. IQR 방식 적용.
ash: 비대칭 분포로 확인됨. IQR 방식 적용.
water: 비대칭 분포로 확인됨. IQR 방식 적용.
superplastic: 비대칭 분포로 확인됨. IQR 방식 적용.
coarseagg: 비대칭 분포로 확인됨. IQR 방식 적용.
fineagg: 비대칭 분포로 확인됨. IQR 방식 적용.
age: 비대칭 분포로 확인됨. IQR 방식 적용.
strength: 비대칭 분포로 확인됨. IQR 방식 적용.
cement: 값 범위 중요 → MinMaxScaler 적용
slag: 값 범위 중요 → MinMaxScaler 적용
ash: 값 범위 중요 → MinMaxScaler 적용
water: 값 범위 중요 → MinMaxScaler 적용
superplastic: 값 범위 중요 → MinMaxScaler 적용
coarseagg: 값 범위 중요 → MinMaxScaler 적용
fineagg: 값 범위 중요 → MinMaxScaler 적용
age: 비대칭 분포 → RobustScaler 적용
strength: 값 범위 중요 → MinMaxScaler 적용


In [9]:
# if target not in ret_dict['processed_df'].columns: raise Exception('unexpected target')
args = argparse.Namespace(
    target = [target],
    data_path='./temp.csv',
    model=model,
    prj_id = 40,
    seed=40
)
surrogate_model.main(args, ret_dict['scalers'])

2025-02-13 15:10:12,833 - INFO - X_train.shape: (740, 8)
2025-02-13 15:10:12,834 - INFO - X_test.shape: (186, 8)
2025-02-13 15:10:12,834 - INFO - y_train.shape: (740, 1)
2025-02-13 15:10:12,835 - INFO - y_test.shape: (186, 1)
c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(186, 1)
(186, 1)
       rmse      mae        r2    target
0  0.055978  0.03315  0.927356  strength
y_test.shape (186, 1)
y_pred.shape (186, 1)


In [10]:
if target not in df.columns: raise Exception('unexpected target')
control_range = df[control_name]
control_range = {col: (df[col].min(), df[col].max()) for col in control_range}

In [11]:
print(target, df[target].min(),'~',df[target].max())

strength 2.33 ~ 82.6


In [12]:
df_scaled = load_data(args.data_path)
X = df_scaled.drop(columns=args.target)
y = df_scaled[args.target]
x_col_list = X.columns.tolist()

X_train,y_train = X.to_numpy(),y.to_numpy()


In [13]:
scalers = ret_dict['scalers']


In [14]:
from src.datasets.data_loader import load_data

In [17]:
for l in [control_name, importance, optimize]:
    for c in l:
        if c not in ret_dict['processed_df'].columns:
            raise Exception(f'{c} is unexpected col_name')
args = argparse.Namespace(
    model=model,
    search_model='k_means',
    data_path='./temp.csv',
    control_name=control_name,
    control_range= control_range,
    target=[target],
    importance=importance,
    optimize=optimize,
    prj_id=40,
    seed=40,
    user_request_target=[user_request_target],
    model_path='./temp/surrogate_model/model',
    user_request_idx=10,
    user_request_value = None
)

In [18]:
# %%time 
x_opt = search_model.main(args,ret_dict['scalers'])

c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\seong\conda3\envs\hack\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\s

TabPFNRegressor(device='cuda')
is_norminal [False, False, False, False, False, False, False, False]
x_min :  [ 0.          0.          0.         -0.05148515  0.          0.
 -0.03235294 -1.28571429]
x_max :  [ 1.          1.05057001  1.          1.18811881  1.45701357  1.
  1.14       16.04761905]
weights (1.0, -1.0, -1.0)


1it [00:03,  3.21s/it]


search model 소요 시간: 3.2184초


In [19]:
df

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29
...,...,...,...,...,...,...,...,...,...
1025,135.0,0.0,166.0,180.0,10.0,961.0,805.0,28,13.29
1026,531.3,0.0,0.0,141.8,28.2,852.1,893.7,3,41.30
1027,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1028,342.0,38.0,0.0,228.0,0.0,932.0,670.0,270,55.06


In [20]:
x_opt

,column_name,ground_truth,predicted
0,cement,[151.79999999999998],[269.41055617710555]
1,slag,[178.1],[101.11839441395027]
2,ash,[138.7],[165.13523971747122]
3,water,[167.5],[221.50200669061906]
4,superplastic,[18.3],[26.497053492235757]
5,coarseagg,[944.0],[802.0282118933405]
6,fineagg,[694.6],[898.4936269760229]
7,age,[28.0],[113.64094626295831]
8,strength,[36.35],[40.01401901245117]


In [21]:
x_opt.iloc[1]

column_name                     slag
ground_truth                 [178.1]
predicted       [101.11839441395027]
Name: 1, dtype: object